In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data_loader import *

# Load files (Matrix, Links & Articles)

In [2]:
names_articles = read_articles()
len(names_articles)
names_articles

0           Áedán mac Gabráin
1                       Åland
2               Édouard Manet
3                        Éire
4       Óengus I of the Picts
                ...          
4599                  Zionism
4600                Zirconium
4601                Zoroaster
4602             Zuid-Gelders
4603                     Zulu
Name: article, Length: 4604, dtype: object

In [3]:
links = read_links()
links

,linkSource,linkTarget
0,Áedán mac Gabráin,Bede
1,Áedán mac Gabráin,Columba
2,Áedán mac Gabráin,Dál Riata
3,Áedán mac Gabráin,Great Britain
4,Áedán mac Gabráin,Ireland
...,...,...
119877,Zulu,South Africa
119878,Zulu,Swaziland
119879,Zulu,United Kingdom
119880,Zulu,Zambia


In [4]:
matrix = read_matrix()
matrix

The rows are the source articles and the columns are the destination articles


article,Áedán mac Gabráin,Åland,Édouard Manet,Éire,Óengus I of the Picts,€2 commemorative coins,10th century,11th century,12th century,13th century,...,Ziad Jarrah,Zimbabwe,Zinc,Zinc chloride,Zion National Park,Zionism,Zirconium,Zoroaster,Zuid-Gelders,Zulu
article,,,,,,,,,,,,,,,,,,,,,
Áedán mac Gabráin,0.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.0,...,4.0,3.0,3.0,4.0,4.0,3.0,4.0,4.0,4.0,2.0
Åland,NaN,0.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,...,4.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0
Édouard Manet,NaN,NaN,0.0,NaN,NaN,NaN,3.0,3.0,2.0,2.0,...,4.0,3.0,2.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0
Éire,NaN,NaN,NaN,0.0,NaN,NaN,3.0,3.0,3.0,3.0,...,4.0,2.0,2.0,3.0,4.0,3.0,4.0,4.0,3.0,3.0
Óengus I of the Picts,NaN,NaN,NaN,NaN,0.0,NaN,2.0,2.0,3.0,2.0,...,4.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zionism,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,...,3.0,2.0,2.0,3.0,3.0,0.0,3.0,3.0,3.0,2.0
Zirconium,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.0,...,3.0,3.0,2.0,2.0,3.0,3.0,0.0,3.0,4.0,3.0
Zoroaster,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,...,3.0,2.0,2.0,3.0,4.0,3.0,3.0,0.0,3.0,3.0


# InDegree

## Option A: With links (faster & safer)
- Numbers of articles != Numbers of articles with a least one **link** directing to it

In [5]:
link_density = links.groupby('linkTarget').count()
link_density = link_density.rename(columns={"linkSource": "indegree"})
link_density

,indegree
linkTarget,
1 Ceres,12
10th century,65
11th century,57
12th century,85
13th century,84
...,...
Zirconium,15
Zoroaster,9
Zuid-Gelders,5


## Option B: With the Matrix (slower & need to carefully distinc between Source & Target)
- Links are not **bidirectional** !
- But allow to also have more info about the best paths possibles

In [6]:
# Define distances to analyze
distance_thresholds = range(1, 10)  # Distances to compute related articles for

# Initialize a dictionary to store counts for each article
related_articles_count = {article: {distance: 0 for distance in distance_thresholds} for article in matrix.columns}

# Count related articles within each distance threshold
for article in matrix.columns:
    for distance in distance_thresholds:
        count = matrix.T.loc[article].apply(lambda x: 1 if x == distance else 0).sum()
        related_articles_count[article][distance] = count

# Convert to a DataFrame for readability
related_articles_df = pd.DataFrame(related_articles_count).T
#related_articles_df.index = names_articles


print("Number of articles at a given distance with the optimal path from the Source to the Target article")
related_articles_df

Number of articles at a given distance with the optimal path from the Source to the Target article


,1,2,3,4,5,6,7,8,9
Áedán mac Gabráin,0,0,0,0,0,0,0,0,0
Åland,0,0,0,0,0,0,0,0,0
Édouard Manet,0,0,0,0,0,0,0,0,0
Éire,0,0,0,0,0,0,0,0,0
Óengus I of the Picts,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
Zionism,25,1020,3406,133,0,0,0,0,0
Zirconium,15,253,1636,2624,55,1,0,0,0
Zoroaster,9,492,3539,541,3,0,0,0,0
Zuid-Gelders,5,98,2723,1737,20,1,0,0,0


In [7]:
# Indegree of the target node
print("The number of pages that have the target node as a link.")
related_articles_df[1]

The number of pages that have the target node as a link.


Áedán mac Gabráin         0
Åland                     0
Édouard Manet             0
Éire                      0
Óengus I of the Picts     0
                         ..
Zionism                  25
Zirconium                15
Zoroaster                 9
Zuid-Gelders              5
Zulu                     14
Name: 1, Length: 4604, dtype: int64

# Link Density (Unique links)
- May be more links per Article, if the same link repeat again througth the artcile, need to confirm by looking at the text or HTML
- Numbers of articles != Numbers of articles with a least one **link** to another article in the corpus

In [8]:
# Group by linkSource and count the number of links to each target
link_density = links.groupby('linkSource').count()
link_density.columns = ['linkDensity']
link_density

,linkDensity
linkSource,
1 Ceres,32
10th century,26
11th century,48
12th century,45
13th century,34
...,...
Åland,19
Édouard Manet,20
Éire,8


In [9]:
# I want to check if by grouping the links by the source, each target is unique
# If the number of links is equal to the number of unique targets, then each target is unique
link_density['uniqueTargets'] = links.groupby('linkSource')['linkTarget'].nunique()

# Check if the two columns are equal
link_density['linkDensity'].equals(link_density['uniqueTargets'])

True